In [1]:
from __future__ import division
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import cPickle as pickle



#read train test data
X_train=pd.read_csv("data/X_train.csv")
y_train=pd.read_csv("data/y_train.csv")
X_test=pd.read_csv("data/X_test.csv")


(78725, 6)

In [2]:
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')
orders_priors = pd.read_csv('data/order_products__prior.csv')
departments = pd.read_csv('data/departments.csv')
aisles = pd.read_csv('data/aisles.csv')

(3346083, 6)
(32434489, 4)


In [3]:
# convert categorical data to dummy values
X_train= pd.get_dummies(X_train, prefix=["d", "h"], columns=['order_dow', 'order_hour_of_day'])

(78725, 35)

In [4]:
# products in department 7
products_bev = products[products['department_id']==7]
products_bev_id = list(products_bev.product_id.values)
products_bev.head(2)

,product_id,product_name,aisle_id,department_id
2,3,Robust Golden Unsweetened Oolong Tea,94,7
6,7,Pure Coconut Water With Orange,98,7


In [5]:
# order priors of department 7
orders_priors_bev = orders_priors[orders_priors['product_id'].isin(products_bev_id)]
orders_prior_id_bev = list(orders_priors_bev.order_id.values)

(2690129, 4)

In [6]:
# orders of department 7
orders_bev = orders[orders['order_id'].isin(orders_prior_id_bev)]

(1457351, 6)

In [7]:
# count orders of department 7 for every user
count_ord_bev = orders_bev.groupby('user_id').count()
count_ord_bev = count_ord_bev['order_id']
count_ord_bev= count_ord_bev.reset_index()
count_ord_bev.rename(columns={'order_id':'order_count'}, inplace=True)

(172795, 2)

In [8]:
X_train = X_train.merge(count_ord_bev, on='user_id', how='left')
X_train = X_train.fillna(0.0)

(78725, 36)

In [10]:
#average interval days per user
average_order_days = orders_bev.groupby('user_id')['days_since_prior_order'].mean().reset_index()
average_order_days.rename(columns={'days_since_prior_order':'avg_days_since_prior'}, inplace=True)

(172795, 2)

In [11]:
X_train = X_train.merge(average_order_days, on='user_id', how='left')
X_train = X_train.fillna(0.0)

(78725, 37)

In [12]:
# beverages per basket per user
#orders_priors['size_of_order']=orders_priors.groupby('order_id')['add_to_cart_order'].transform('max')
#orders_priors.shape
#orders_priors['total_beverages']=orders_priors.groupby('order_id')['product_id'].transform(lambda x: (x.isin(products_bev_id)).sum())

In [13]:
#orders_priors['bev_per_basket']= np.where(orders_priors['total_beverages'] < 1, orders_priors['total_beverages'], np.around(orders_priors['total_beverages']/orders_priors['size_of_order'], decimals=2))


In [14]:
#pickle.dump(orders_priors, open("orders_priors.pkl", 'wb'))

In [15]:
orders_priors = pd.read_pickle("orders_priors.pkl")

In [17]:
orders_priors_bev = orders_priors[orders_priors['product_id'].isin(products_bev_id)]
orders_prior_id_bev = list(orders_priors_bev.order_id.values)
orders_bev = orders[orders['order_id'].isin(orders_prior_id_bev)]

orders_bev = orders_bev.merge(orders_priors_bev[['order_id','bev_per_basket']].drop_duplicates(subset=['order_id']), on='order_id', how='left')


In [19]:
bev_per_basket = orders_bev.groupby('user_id')['bev_per_basket'].mean().reset_index()

(172795, 2)

In [20]:
X_train = X_train.merge(bev_per_basket, on='user_id', how='left')
X_train  = X_train.fillna(0.0)

(78725, 38)

In [23]:
X_train = X_train.drop(['order_id','user_id'], axis=1)

In [24]:
X_example_train, X_example_test, y_example_train, y_example_test = train_test_split(X_train, y_train, test_size=0.33)
logreg = LogisticRegression()
logreg.fit(X_example_train, y_example_train["category"])
y_pred = logreg.predict(X_example_test)
print("logreg",accuracy_score(y_example_test["category"], y_pred))

('logreg', 0.57948421862971522)
